In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [28]:
def inverted_residual_block(inputs, filters, expansion_factor, strides):
    in_channels = inputs.shape[-1]
    
    x = layers.Conv2D(in_channels * expansion_factor, kernel_size=1, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    
    if in_channels == filters and strides == 1:
        return layers.Add()([x, inputs])
    return x

In [29]:
def build_mobilenet_v2(input_shape=(224, 224, 3), num_classes=1000):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = inverted_residual_block(x, filters=16, expansion_factor=1, strides=1)
    
    x = inverted_residual_block(x, filters=24, expansion_factor=6, strides=2)
    x = inverted_residual_block(x, filters=24, expansion_factor=6, strides=1)
    
    x = inverted_residual_block(x, filters=32, expansion_factor=6, strides=2)
    x = inverted_residual_block(x, filters=32, expansion_factor=6, strides=1)
    x = inverted_residual_block(x, filters=32, expansion_factor=6, strides=1)
    
    x = inverted_residual_block(x, filters=64, expansion_factor=6, strides=2)
    x = inverted_residual_block(x, filters=64, expansion_factor=6, strides=1)
    x = inverted_residual_block(x, filters=64, expansion_factor=6, strides=1)
    x = inverted_residual_block(x, filters=64, expansion_factor=6, strides=1)
    
    x = inverted_residual_block(x, filters=96, expansion_factor=6, strides=1)
    x = inverted_residual_block(x, filters=96, expansion_factor=6, strides=1)
    x = inverted_residual_block(x, filters=96, expansion_factor=6, strides=1)
    
    x = inverted_residual_block(x, filters=160, expansion_factor=6, strides=2)
    x = inverted_residual_block(x, filters=160, expansion_factor=6, strides=1)
    x = inverted_residual_block(x, filters=160, expansion_factor=6, strides=1)
    
    x = inverted_residual_block(x, filters=320, expansion_factor=6, strides=1)
    
    x = layers.Conv2D(1280, kernel_size=1, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, x)
    return model

In [ ]:
mobilenetv2_scratch_model = build_mobilenet_v2()
mobilenetv2_scratch_model.summary()

In [ ]:
mobilenetv2_tf_model = MobileNetV2(include_top=True, weights="imagenet", input_shape=(224, 224, 3))
mobilenetv2_tf_model.summary()